In [1]:
from src.data import DataModule

dm = DataModule(
    r"C:\Users\user\data\dl_radiomics\preprocessed_3d",
    r"C:\Users\user\data\tables\lesion_followup_curated_v4.csv",
    "lung",
    "mst",
)

Loading dataset: 100%|██████████| 56/56 [00:09<00:00,  5.79it/s]


In [3]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, Trainer, seed_everything
from monai.networks.nets.densenet import DenseNet121

class Model(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = DenseNet121(spatial_dims=3, in_channels=1, out_channels=1)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=1e-3)

        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = torch.sigmoid(self.model(x))

        loss = nn.BCELoss()(y_hat.squeeze(), y.float())

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = torch.sigmoid(self.model(x))

        loss = nn.BCELoss()(y_hat.squeeze(), y.float())

        return loss


model = Model()

seed_everything(0)

trainer = Trainer(
    max_epochs=150,
    gpus=1,
    deterministic=True,
    fast_dev_run=False
)
        
trainer.fit(model, dm)

    


Global seed set to 0


2022-10-26 11:56:50,922 - Global seed set to 0


GPU available: True, used: True


2022-10-26 11:56:50,924 - GPU available: True, used: True


TPU available: None, using: 0 TPU cores


2022-10-26 11:56:50,925 - TPU available: None, using: 0 TPU cores
2022-10-26 11:56:50,927 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type        | Params
--------------------------------------
0 | model | DenseNet121 | 11.2 M
--------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.975    Total estimated model params size (MB)


2022-10-26 11:56:51,169 - 
  | Name  | Type        | Params
--------------------------------------
0 | model | DenseNet121 | 11.2 M
--------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.975    Total estimated model params size (MB)
Epoch 149: 100%|██████████| 6/6 [00:12<00:00,  2.14s/it, loss=0.321, v_num=0]


1